In [1]:
import pulp
import numpy as np
import math

np.random.seed(999)

### Dados

In [2]:
num_pessoas = 20
num_equipes = 4
conhecimento_min = 1
conhecimento_max = 10
t_min = 4
t_max = 6
prob_alta_afinidade = 0.3
prob_seniors = 0.15  
penalidade = 15.0 # penalização por não ter sênior

In [3]:
# Conhecimento de cada pessoa em cada framework
conhecimento = np.random.randint(conhecimento_min, conhecimento_max+1, size=(num_pessoas, num_equipes))

# Pares com afinidade muito alta que não podem estar na mesma equipe
alta_afinidade = np.random.choice([0, 1], size=(num_pessoas, num_pessoas), p=[1 - prob_alta_afinidade, prob_alta_afinidade])
np.fill_diagonal(alta_afinidade, 0)
for i in range(num_pessoas):
    for j in range(i+1, num_pessoas):
        alta_afinidade[j, i] = alta_afinidade[i, j]

# senior[i] = 1 se i for senior
num_seniors = math.floor(prob_seniors * num_pessoas)
senior = np.zeros(num_pessoas, dtype=int)
seniors_indices = np.random.choice(num_pessoas, size=num_seniors, replace=False)
senior[seniors_indices] = 1

In [4]:
model = pulp.LpProblem("hackathon", pulp.LpMinimize)

### Variáveis

In [5]:
# Xij = Pessoa i na equipe j? 
x = pulp.LpVariable.dicts("x", ((i,j) for i in range(num_pessoas) for j in range(num_equipes)), cat='Binary')

# Sj = Equipe j possui algum senior ?
s = pulp.LpVariable.dicts("s", (j for j in range(num_equipes)), cat='Binary')

# Pj = Pontuação da equipe j
P = pulp.LpVariable.dicts("P", (j for j in range(num_equipes)), lowBound=0, cat='Continuous')

# Pontuações das equipes com maior e menor pontuação
P_max = pulp.LpVariable("P_max", lowBound=0, cat='Continuous')
P_min = pulp.LpVariable("P_min", lowBound=0, cat='Continuous')

### Restrições

In [9]:
# Cada pessoa deve estar em exatamente uma equipe
for i in range(num_pessoas):
    model += pulp.lpSum(x[i,j] for j in range(num_equipes)) == 1

# Tamanho da equipe entre t_min e t_max
for j in range(num_equipes):
    model += pulp.lpSum(x[i,j] for i in range(num_pessoas)) >= t_min
    model += pulp.lpSum(x[i,j] for i in range(num_pessoas)) <= t_max

# Proibir pares com afinidade alta de estarem na mesma equipe
for j in range(num_equipes):
    for i in range(num_pessoas):
        for k in range(i+1, num_pessoas):
            if alta_afinidade[i, k] == 1:
                model += x[i, j] + x[k, j] <= 1

# s_j = 1 se equipe j tiver um sênior
for j in range(num_equipes):
    model += s[j] == pulp.lpSum(senior[i] * x[i,j] for i in range(num_pessoas))
    model += pulp.lpSum(senior[i] * x[i,j] for i in range(num_pessoas)) <= 1 # max 1 senior por equipe

# Calcular pontuação Pj
for j in range(num_equipes):
    conhecimento_total = pulp.lpSum(conhecimento[i, j] * x[i, j] for i in range(num_pessoas))
    model += P[j] == conhecimento_total - penalidade * (1 - s[j])

# Objetivo é minimizar P_max - P_min
for j in range(num_equipes):
    model += P[j] <= P_max
    model += P[j] >= P_min


In [7]:
model += P_max - P_min
solver = pulp.PULP_CBC_CMD(msg=True)
model.solve(solver)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/andre/python/pesquisa_operacional/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/80781dd9958e4a76a11bf8c806b4a278-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/80781dd9958e4a76a11bf8c806b4a278-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 281 COLUMNS
At line 1280 RHS
At line 1557 BOUNDS
At line 1642 ENDATA
Problem MODEL has 276 rows, 90 columns and 828 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 93 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 72 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 23 strengthened rows, 0 substitutions
Cgl0004I processed model has 190 rows, 86 columns (84 integer (84 of which binary)) an

1

In [8]:
print(f"Status: {pulp.LpStatus[model.status]}")
print(f"Diferença entre scores máximo e mínimo: {pulp.value(P_max) - pulp.value(P_min):.2f}\n")

pontuacoes = []
for j in range(num_equipes):
    equipe = [i for i in range(num_pessoas) if pulp.value(x[i, j]) > 0.5]
    pontuacao = pulp.value(P[j])
    pontuacoes.append(pontuacao)
    tem_senior = pulp.value(s[j]) > 0.5
    conhecimento_total = 0
    
    print(f"\nEquipe {j + 1}:")
    print(f"  Membros: {equipe}")
    print(f"  Detalhes dos membros:")
    
    for i in equipe:
        eh_senior = " (Sênior)" if senior[i] else ""
        c = conhecimento[i][j]
        conhecimento_total += c
        print(f"    - Pessoa {i}: Conhecimento = {c} {eh_senior}")

    # print(f"  Afinidades entre membros (deve ser 0):")
    # for i in range(len(equipe)):
    #     for k in range(i+1, len(equipe)):
    #         a = alta_afinidade[equipe[i], equipe[k]]
    #         print(f"    - ({equipe[i]}, {equipe[k]}): {a}")

    penalidade_total = penalidade if not tem_senior else 0
    pontuacao_calculada = conhecimento_total - penalidade_total

    print(f"  Tem sênior? {'Sim' if tem_senior else 'Não'}")
    print(f"  Soma dos conhecimentos: {conhecimento_total}")
    print(f"  Penalidade por falta de sênior: {penalidade_total}")
    # print(f"  Pontuação calculada (conhecimentos - penalidade): {pontuacao_calculada:.2f}")
    print(f"  Pontuação (modelo): {pontuacao:.2f}")

print(f"\nMaior score entre as equipes: {max(pontuacoes):.2f}")
print(f"Menor score entre as equipes: {min(pontuacoes):.2f}")

# print(f"\nÍndices dos seniors: {seniors_indices.tolist()}")


Status: Optimal
Diferença entre scores máximo e mínimo: 2.00


Equipe 1:
  Membros: [4, 5, 11, 17]
  Detalhes dos membros:
    - Pessoa 4: Conhecimento = 5 
    - Pessoa 5: Conhecimento = 7  (Sênior)
    - Pessoa 11: Conhecimento = 7 
    - Pessoa 17: Conhecimento = 7 
  Tem sênior? Sim
  Soma dos conhecimentos: 26
  Penalidade por falta de sênior: 0
  Pontuação (modelo): 26.00

Equipe 2:
  Membros: [8, 9, 10, 13, 15, 16]
  Detalhes dos membros:
    - Pessoa 8: Conhecimento = 8 
    - Pessoa 9: Conhecimento = 2 
    - Pessoa 10: Conhecimento = 2  (Sênior)
    - Pessoa 13: Conhecimento = 7 
    - Pessoa 15: Conhecimento = 7 
    - Pessoa 16: Conhecimento = 1 
  Tem sênior? Sim
  Soma dos conhecimentos: 27
  Penalidade por falta de sênior: 0
  Pontuação (modelo): 27.00

Equipe 3:
  Membros: [1, 14, 18, 19]
  Detalhes dos membros:
    - Pessoa 1: Conhecimento = 4 
    - Pessoa 14: Conhecimento = 10 
    - Pessoa 18: Conhecimento = 8 
    - Pessoa 19: Conhecimento = 6  (Sênior)
  Tem sênio